In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

def logic_win(x,x_r):
    return (x >= x_r[0]) & (x < x_r[1])

#### Import and clean dataset, select out London

In [2]:
path = 'combined_collisions_v3.csv'

df = pd.read_csv(path,low_memory=False)
df.drop(columns='Unnamed: 0',inplace=True)

In [3]:
#Time column has some nans in it, must drop those entries

def is_str(x):
    Nx = len(x)
    I = np.ones(Nx).astype(bool)
    for ii in range(Nx):
        if type(x[ii]) != str:
            I[ii] = False
    return I

df = df.loc[is_str(df.Time.values),:]
df.aadf_Count_point_id = df.aadf_Count_point_id.values.astype(int)

In [4]:
#make hour, year and month columns
    #hour will be rounded 
def get_hour(T):
    Nt = len(T)
    H = np.zeros(Nt).astype(int)
    for ii in range(Nt):
        s = T[ii].split(':')
        h = int(s[0])
        m = int(s[1])
        if m > 30:
            h += 1
        H[ii] = h % 24
    return H

def get_ymd(D):
    Nd = len(D)
    # |
    ymd = np.zeros((Nd,3)).astype(int)
    for ii in range(Nd):
        s = D[ii].split('-')
        for nn in range(3):
            ymd[ii,nn] = int(s[nn])
    return ymd

def reformat_aadf_year(Year):
    Ny = len(Year)
    y = np.zeros(Ny).astype(int)
    for ii in range(Ny):
        y[ii] = int(Year[ii].split('-')[0])
    return y

df[['y','m','d']] = get_ymd(df.Date.values)
df['h'] = get_hour(df.Time.values)
#df['aadf_Year'] = reformat_aadf_year(df.aadf_Year.values)

##### Below selects out region. You can use Dan's feature to select Greater London, or code below for central London.
- additionally I drop all entries with no traffic counter matches

In [5]:
#central London box
Lat_r = [51.475,51.543]
Lon_r = [-0.179,0.023]

I_lat = (df.Latitude > Lat_r[0]) & (df.Latitude < Lat_r[1])
I_lon = (df.Longitude > Lon_r[0]) & (df.Longitude < Lon_r[1])
I = I_lat & I_lon

#df_lond = df.loc[I,:].copy() #selects central London
df_lond = df.loc[df.in_london,:].copy() #this selects Greater London

In [10]:
#drop all data that don't have traffic matches

I = df_lond.match == True
df_lond = df_lond.loc[I,:]
df_lond.aadf_Year = reformat_aadf_year(df_lond.aadf_Year.values)

#### Make neighborhoods feature

In [11]:
#Make a 3 (Lat, N/S) x 6 (Lon, E/W) grid
Nlat = 3
Nlon = 6
Nneigh = Nlon*Nlat

edges_lat = np.linspace(*Lat_r,Nlat+1)
dlat = np.diff(edges_lat)[0]
bins_lat = edges_lat[:-1]+0.5*dlat
edges_lon = np.linspace(*Lon_r,Nlon+1)
dlon = np.diff(edges_lon)[0]
bins_lon = edges_lon[:-1]+0.5*dlon

coord_neigh = [None]*Nneigh

df_lond['neigh'] = 0 #initialize neighborhood feature
ir = 0 #these are lat bins (rows)
ic = 0 #lon bins
for ii in range(Nneigh):
    Ilat = logic_win(df_lond.Latitude,(edges_lat[ir],edges_lat[ir+1]))
    Ilon = logic_win(df_lond.Longitude,(edges_lon[ic],edges_lon[ic+1]))
    I = Ilat & Ilon
    df_lond.loc[I,'neigh'] = ii
    
    coord_neigh[ii] = np.array([bins_lat[ir],bins_lon[ic]])
    
    ic+=1
    if ic == Nlon:
        ic = 0
        ir+=1

In [35]:
list(df_lond.columns)

['Accident_Index',
 'Location_Easting_OSGR',
 'Location_Northing_OSGR',
 'Longitude',
 'Latitude',
 'Police_Force',
 'Accident_Severity',
 'Number_of_Vehicles',
 'Number_of_Casualties',
 'Date',
 'Day_of_Week',
 'Time',
 'Local_Authority_(District)',
 'Local_Authority_(Highway)',
 '1st_Road_Class',
 '1st_Road_Number',
 'Road_Type',
 'Speed_limit',
 'Junction_Detail',
 'Junction_Control',
 '2nd_Road_Class',
 '2nd_Road_Number',
 'Pedestrian_Crossing-Human_Control',
 'Pedestrian_Crossing-Physical_Facilities',
 'Light_Conditions',
 'Weather_Conditions',
 'Road_Surface_Conditions',
 'Special_Conditions_at_Site',
 'Carriageway_Hazards',
 'Urban_or_Rural_Area',
 'Did_Police_Officer_Attend_Scene_of_Accident',
 'LSOA_of_Accident_Location',
 'Vehicle_Type',
 'Bicycles',
 'fatal',
 'serious',
 'slight',
 'geometry',
 'index_right',
 'Road_name',
 'Road_name2',
 'in_london',
 'aadf_Count_point_id',
 'aadf_Year',
 'aadf_Region_id',
 'aadf_Region_name',
 'aadf_Region_ons_code',
 'aadf_Local_authorit

In [38]:
Is = [np.array([True,True,True]),np.array([False,True,False]),np.array([True,True,False])]

Itrue = np.ones(3).astype(bool)
for I in Is:
    Itrue &= I
    
Itrue

array([False,  True, False])

In [41]:
#Make a feature for if there is a bike lane of not ('bikelane'). There is no bike lane if all
#the below classifiers are False

f_laneClass = ['aadf_CLT_CARR', 'aadf_CLT_SEGREG', 'aadf_CLT_STEPP', 'aadf_CLT_PARSEG',
       'aadf_CLT_SHARED', 'aadf_CLT_MANDAT', 'aadf_CLT_ADVIS',
       'aadf_CLT_PRIORI', 'aadf_CLT_CONTRA', 'aadf_CLT_BIDIRE',
       'aadf_CLT_CBYPAS', 'aadf_CLT_BBYPAS', 'aadf_CLT_PARKR',
       'aadf_CLT_WATERR', 'aadf_CLT_PTIME']

Inolane = np.ones(df_lond.shape[0]).astype(bool)
for f in f_laneClass:
    Inolane &= (df_lond[f].values == False)
    
df_lond['bikelane'] = ~Inolane

In [45]:
traf_IDs = np.array(df_lond.aadf_Count_point_id.value_counts().index)
Ninc_IDs = np.array(df_lond.aadf_Count_point_id.value_counts().values)

######################################
Nthresh = 1 #this sets which traffic counters are included, they must have at least this many accidents 
######################################

I = Ninc_IDs >= Nthresh #this will select roads to include in the new DataFrame based on if they had at least this many accidents
I.sum()

IDs_df = traf_IDs[I]
Nid = len(IDs_df)

df_road = pd.DataFrame({'ID':IDs_df})
slight_rd = np.zeros(Nid).astype(int)
ser_rd = np.zeros(Nid).astype(int)
total_rd = np.zeros(Nid).astype(int)
for nn,ii in enumerate(IDs_df):
    I = df_lond.aadf_Count_point_id.values == ii
    total_rd[nn] = I.sum()
    slight_rd[nn] = df_lond.loc[I,'slight'].values.sum()
    ser_rd[nn] = total_rd[nn] - slight_rd[nn]
df_road['total'] = total_rd
df_road['slight'] = slight_rd
df_road['serious'] = ser_rd

safety_mode = ['neigh','Road_name','Road_name2','h','Day_of_Week','m','Speed_limit','Junction_Detail','Junction_Control','Pedestrian_Crossing-Physical_Facilities']
aadf_av = ['distance_to_cp','aadf_Pedal_cycles','aadf_Pedal_cycles','aadf_All_motor_vehicles','aadf_All_HGVs','aadf_LGVs','aadf_Buses_and_coaches']
aadf_mode = ['bikelane','aadf_Road_name','aadf_Road_category','aadf_Road_type','aadf_CLT_CARR', 'aadf_CLT_SEGREG', 'aadf_CLT_STEPP', 'aadf_CLT_PARSEG',
       'aadf_CLT_SHARED', 'aadf_CLT_MANDAT', 'aadf_CLT_ADVIS',
       'aadf_CLT_PRIORI', 'aadf_CLT_CONTRA', 'aadf_CLT_BIDIRE',
       'aadf_CLT_CBYPAS', 'aadf_CLT_BBYPAS', 'aadf_CLT_PARKR',
       'aadf_CLT_WATERR', 'aadf_CLT_PTIME','aadf_Link_length_miles']

for f in safety_mode:
    df_road[f] = np.zeros(Nid).astype(int)
    for nn,ii in enumerate(IDs_df):
        I = df_lond.aadf_Count_point_id.values == ii
        df_road.loc[nn,f] = df_lond.loc[I,f].mode().values[0]

for f in aadf_mode:
    df_road[f] = np.zeros(Nid).astype(int)
    for nn,ii in enumerate(IDs_df):
        I = df_lond.aadf_Count_point_id.values == ii
        df_road.loc[nn,f] = df_lond.loc[I,f].mode().values[0]
        
for f in aadf_av:
    df_road[f] = np.zeros(Nid).astype(int)
    for nn,ii in enumerate(IDs_df):
        I = df_lond.aadf_Count_point_id.values == ii
        df_road.loc[nn,f] = df_lond.loc[I,f].mean()
        
median_d = df_road.distance_to_cp.median()
median_bv = df_road.aadf_Pedal_cycles.median()
df_road['total_dnorm'] = df_road['total'].values*median_d/df_road['distance_to_cp'].values
df_road['total_dnorm_bvnorm'] = df_road.total_dnorm.values*median_bv/df_road.aadf_Pedal_cycles.values
df_road['serious_ratio'] = (df_road['serious'].values)/df_road['total'].values

<ipython-input-45-9f85eff6dc66>:56: RuntimeWarning: divide by zero encountered in divide
  df_road['total_dnorm_bvnorm'] = df_road.total_dnorm.values*median_bv/df_road.aadf_Pedal_cycles.values


In [46]:
#add in aadf year availability metadata 
path_year = 'aadf_year_meta.csv'
df_year = pd.read_csv(path_year)

#merge the metadata into df_road
feat_add = list(df_year.columns)
feat_add.remove('Count_point_id')
for c in feat_add:
    df_road[c] = np.zeros(Nid).astype(int)
    for ii,ID in enumerate(df_road.ID.values):
        I = df_year.Count_point_id == ID
        df_road.loc[ii,c] = df_year.loc[I,c].values

#### Here is the final DataFrame, feel free to save your own version

In [47]:
df_road.sample(20)

,ID,total,slight,serious,neigh,Road_name,Road_name2,h,Day_of_Week,m,...,aadf_All_HGVs,aadf_LGVs,aadf_Buses_and_coaches,total_dnorm,total_dnorm_bvnorm,serious_ratio,N_year,Year_start,Year_end,Max_year_gap
1078,46663,8,6,2,0,A109,A10,13,1,7,...,502.625000,1934.125000,796.875000,7.369899,9.461082,0.250000,22,2000,2021,1
225,37708,44,40,4,0,A3218,U,8,5,6,...,274.477273,2273.454545,810.840909,50.114561,7.128727,0.090909,22,2000,2021,1
447,37707,25,23,2,0,A3217,U,18,3,6,...,622.840000,2659.600000,1062.440000,38.419004,5.665731,0.080000,22,2000,2021,1
1496,26673,4,3,1,0,A125,na0,7,4,11,...,772.750000,2561.000000,211.250000,1.423439,6.113570,0.250000,22,2000,2021,1
1194,8087,7,6,1,0,A4089,U,8,4,3,...,154.000000,835.428571,409.142857,6.481186,21.398409,0.142857,22,2000,2021,1
281,28878,36,32,4,4,A1206,U,19,3,5,...,141.638889,1133.277778,675.333333,16.513386,4.004924,0.111111,22,2000,2021,1
1700,8639,2,1,1,0,A307,A308,8,5,8,...,295.000000,1601.500000,240.000000,5.727699,5.526103,0.500000,22,2000,2021,1
883,38560,11,10,1,0,A103,A1201,8,6,4,...,488.545455,4076.272727,1363.909091,33.127234,8.427305,0.090909,22,2000,2021,1
162,56697,52,46,6,0,A315,U,18,4,8,...,547.480769,1663.000000,618.538462,35.098715,13.610371,0.115385,22,2000,2021,1
579,38570,20,16,4,0,A3,A3220,7,3,8,...,953.500000,3291.800000,602.400000,100.873397,15.784226,0.200000,22,2000,2021,1


In [ ]:
##save dataframe

# path = 'df_road_v2.csv'
# df_road.to_csv(path,index_label=False)